In [ ]:
import cv2
import numpy as np
import os
from random import shuffle
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt
import glob as gb
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/rdhenkawat/ovarian-cancer")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: 

Abort: ignored

In [ ]:
cancer="/content/train1/"

In [ ]:
import PIL
from PIL import Image,ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
os.listdir(cancer)
for file in os.listdir(cancer):
  f_img = cancer + file
  for ovar in os.listdir(f_img):
    t_img= f_img + '/' + ovar
    img = Image.open(t_img)
    img = img.resize((128,128))
    img.save(t_img)

In [ ]:
os.listdir("train1")

['Endometri', 'noncancer', '.ipynb_checkpoints']

In [ ]:
import shutil
shutil.rmtree("train1/.ipynb_checkpoints")


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rotation_range=20,
                             shear_range=10,
                             samplewise_center=True,
                             samplewise_std_normalization= True,
                             height_shift_range=10,
                             width_shift_range=30,
                             horizontal_flip=True,
                             vertical_flip=True,
                             validation_split=0.2
                             )
train_generator = datagen.flow_from_directory('/content/train1',
                                              target_size=(128,128),
                                              subset='training',
                                              shuffle=shuffle)

val_generator = datagen.flow_from_directory('/content/train1',
                                            target_size=(128,128),
                                            subset='validation',
                                            shuffle=shuffle)

Found 193 images belonging to 2 classes.
Found 47 images belonging to 2 classes.


In [ ]:
pre_trained_model=tf.keras.applications.DenseNet169(
    include_top=False,
    weights="imagenet",
    input_shape=(128,128,3)
)

51877672/51877672 [==============================] - 0s 0us/step


In [ ]:
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping
from keras import layers


In [ ]:
, learning_rate_schedule
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam

x = tf.keras.layers.Flatten()(pre_trained_model.output)



x=tf.keras.layers.Dense(2, activation='softmax')(x)


model = Model( pre_trained_model.input, x)

print(model.summary())
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc'])
model.optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.0001)
reduce_lr = ReduceLROnPlateau(patience=0,verbose=1,monitor="val_acc")
early_stop = EarlyStopping(patience=2,verbose=1,monitor="val_loss")
filepath="efficient_weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True,mode='max')

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 134, 134, 3)  0          ['input_3[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 64, 64, 64)   9408        ['zero_padding2d[0][0]']         
                                                                                                  
 conv1/bn (BatchNormalization)  (None, 64, 64, 64)   256         ['conv1/conv[0][0]']       

In [ ]:
history = model.fit_generator(train_generator,
                              validation_data=val_generator,
                              epochs=10,
                              callbacks=[checkpoint,reduce_lr,early_stop],
                              verbose=1,steps_per_epoch=5)

<ipython-input-36-8320b7d10ef9>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator,


Epoch 1/10
5/5 [==============================] - ETA: 0s - loss: 0.6743 - acc: 0.6357 
Epoch 1: acc improved from -inf to 0.63566, saving model to efficient_weights.hdf5
5/5 [==============================] - 132s 11s/step - loss: 0.6743 - acc: 0.6357 - val_loss: 165.1128 - val_acc: 0.6383 - lr: 1.0000e-04
Epoch 2/10
5/5 [==============================] - ETA: 0s - loss: 0.1602 - acc: 0.9750 
Epoch 2: acc improved from 0.63566 to 0.97500, saving model to efficient_weights.hdf5

Epoch 2: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
5/5 [==============================] - 59s 12s/step - loss: 0.1602 - acc: 0.9750 - val_loss: 217.3482 - val_acc: 0.6383 - lr: 1.0000e-04
Epoch 3/10
5/5 [==============================] - ETA: 0s - loss: 0.1092 - acc: 0.9750 
Epoch 3: acc did not improve from 0.97500

Epoch 3: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-07.
5/5 [==============================] - 56s 11s/step - loss: 0.1092 - acc: 0.9750 - val_loss: 130